In [1]:
import sys
sys.path.append("../..")
from thesis.utils.utils import setup_ray, save, load, Experiment
path = "D:/Master/Masterarbeit/thesis"
setup_ray(path = path, unidirectional = False, seed=69)

2023-02-28 17:07:11,316	INFO worker.py:1528 -- Started a local Ray instance.


In [2]:
env_args = dict(
    fleetsize = 8,
    max_fleetsize = 20,    
    pseudo_routing = False,
    pseudo_dispatcher = True,
    pseudo_dispatcher_clever = False,
    #pseudo_dispatcher_distance = 0.3,
    routing_agent_death= True,
    death_on_target = False,
    transform_dispatching_partobs = True,
    direction_reward = -0.1,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 10,
        #reward_reached_target_by_time = True, 
        reward_wrong_target = -1,
        reward_removed_for_block = -10, 
        reward_target_distance = 0,
        reward_invalid= -0.1,
        reward_duration = -0.5,
        block_timeout = 120,
        reward_accepted_in_station = 1,
        reward_declined_in_station = -1,
        routing_interval = 2,
        dispatching_interval=360,
        io_quote = 0.99  ,
        availability = 0.95,
        mttr = 5,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "gnn_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = False,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
            env_type = "matrix"
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=64,
            with_action_mask=False,
            with_agvs=True,
            with_stations = True,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
        )
    )
)

In [4]:
exp = Experiment("matrix_routing")
for seed in [42, 43, 44]:
    exp.experiment(
        path = path,
        env_args = env_args, 
        agv_model = agv_model,
        dispatcher_model=dispatcher_model,
        run_name="04_rew_dir_-0_1", 
        env = "matrix",
        algo = "dqn",
        n_intervals =8,
        #train_agv = False,
        backup_interval=50,
        batch_size=300, #apex + gnn: 50
        seed = seed,
        algo_params = {"grad_clip": 1, "gamma":0.9, "exploration_config":{"warmup_timesteps": 0,"epsilon_timesteps": 200000,"final_epsilon": 0.02,"initial_epsilon": 1.0,"type": "EpsilonGreedy"}},# "exploration_config": {"type": "Curiosity", "sub_exploration": {"type": "StochasticSampling"}, "eta": 0.1}},
        lr = 1e-3,
        #load_agv="../../models/matrix_routing/primal_8_20_2023-02-11_23-21-50/checkpoint_000650",
        
    )

2023-02-28 17:07:17,866	INFO simple_q.py:307 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2023-02-28 17:07:17,875	INFO algorithm.py:457 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_71340_upi8vv_3)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_71354_s5gu48j8)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma

In [ ]:
#load(exp.trainer, "agv", "../../models/trained")

In [ ]:
#save(exp.trainer, "agv", "../../models/trained")

In [ ]:
#exp.keep_training(4)

In [ ]:
#exp.keep_training(10)